# 🔄 Silver - tb_motivo (SRC)

**Camada**: Silver  
**Catálogo**: v_credit  
**Tabelas de Saída**:
- `tb_motivo` → Registros válidos
- `tb_motivo_invalidos` → Registros rejeitados (auditoria)

**Origem**: `v_credit.bronze.base_motivos`

**Objetivo**: Limpar, padronizar e validar dados de motivos com governança de qualidade

## 📋 Imports e Configuração

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Capturar timestamp único no início do processamento
# Isso garante que todos os registros terão o mesmo timestamp
processing_timestamp = spark.sql("SELECT current_timestamp() as ts").collect()[0]['ts']

print("="*80)
print(f"🚀 Processamento Silver - tb_motivo")
print("="*80)
print(f"Catálogo: v_credit")
print(f"Estratégia: Merge + Auditoria de Inválidos")
print(f"Timestamp do processamento: {processing_timestamp}")
print("="*80)


## 1️⃣ Leitura da Bronze

In [0]:
# Configuração
TABLE_SOURCE = "v_credit.bronze.base_motivos"
TABLE_TARGET_VALIDOS = "v_credit.silver.tb_motivo"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_motivo_invalidos"

print(f"📥 Origem: {TABLE_SOURCE}")
print(f"📤 Destino (válidos): {TABLE_TARGET_VALIDOS}")
print(f"📤 Destino (inválidos): {TABLE_TARGET_INVALIDOS}")

In [0]:
# Ler dados da Bronze
df_bronze = spark.table(TABLE_SOURCE)

total_bronze = df_bronze.count()
print(f"\n✅ Total de registros lidos da Bronze: {total_bronze}")

In [0]:
# Visualizar amostra
print("\n📊 Amostra dos dados Bronze:")
display(df_bronze.limit(5))

## 2️⃣ Limpeza e Padronização

In [0]:
# ---------------------------
# Categorização automática
# ---------------------------
df_categorizado = df_bronze.withColumn(
    "cd_categoria_temp",
    F.when(
        F.lower(F.col("nome_motivo")).rlike(
            "cadastr|dados|telefone|email|vencimento|transferência|agência|cancelamento"
        ), 
        1  # CADASTRAL
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "cartão|bloqueio|desbloqueio|adicional|compra|contraç"
        ),
        2  # CARTAO
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "fatura|limite|contrato|renegociação|dívida|pagamento"
        ),
        3  # FINANCEIRO
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "aplicativo|app|site|sistema|erro|problema|técnico"
        ),
        4  # ATENDIMENTO
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "pontos|programa|benefício"
        ),
        5  # BENEFICIOS
    ).otherwise(0)  # NAO_CLASSIFICADO
)

print("✅ Categorização automática aplicada")

In [0]:
df_limpo = (
    df_categorizado
    .select(
        F.col("id_motivo").cast("bigint").alias("cd_motivo"),
        F.trim(F.col("nome_motivo")).alias("ds_motivo"),
        F.col("cd_categoria_temp").cast("bigint").alias("cd_categoria"),
        F.when(
            F.upper(F.trim(F.col("criticidade"))) == "BAIXA", 1
        ).when(
            F.upper(F.trim(F.col("criticidade"))).isin(["MÉDIA", "MEDIA"]), 2
        ).when(
            F.upper(F.trim(F.col("criticidade"))) == "ALTA", 3
        ).otherwise(None)
        .cast("bigint")
        .alias("cd_criticidade"),
        F.coalesce(
            F.col("ingestion_timestamp"),
            F.lit(processing_timestamp)
        ).cast("timestamp").alias("dt_ingestion"),
        F.coalesce(
            F.col("origem"),
            F.lit("base_motivos")
        ).cast("string").alias("dc_origem")
    )
    .dropDuplicates(["cd_motivo"])
    .withColumn("data_processamento", F.lit(processing_timestamp))
)

total_limpo = df_limpo.count()
print(f"✅ Dados limpos e padronizados: {total_limpo} registros")

In [0]:
# Visualizar dados limpos
print("\n📊 Amostra dos dados após limpeza:")
display(df_limpo.limit(5))

## 3️⃣ Regras de Validação e Qualidade

In [0]:
# ---------------------------
# Aplicar regras de validação
# ---------------------------
df_validacao = (df_limpo
    
    # Flag 1: cd_motivo não pode ser NULL
    .withColumn("flag_id_valido", 
        F.col("cd_motivo").isNotNull()
    )
    
    # Flag 2: ds_motivo não pode estar vazio ou NULL
    .withColumn("flag_nome_valido", 
        (F.col("ds_motivo").isNotNull()) & (F.length(F.trim(F.col("ds_motivo"))) > 0)
    )
    
    # Flag 3: cd_categoria deve estar entre 0 e 5
    .withColumn("flag_categoria_valida", 
        F.col("cd_categoria").between(0, 5)
    )
    
    # Flag 4: cd_criticidade deve ser válida (1, 2 ou 3)
    .withColumn("flag_criticidade_valida", 
        F.col("cd_criticidade").isin([1, 2, 3])
    )
    
    # Flag geral de qualidade
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_nome_valido") &
            F.col("flag_categoria_valida") &
            F.col("flag_criticidade_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

print("✅ Regras de validação aplicadas")

In [0]:
# Visualizar com flags
print("\n📊 Amostra com flags de validação:")
display(df_validacao.select(
    "cd_motivo",
    "ds_motivo",
    "cd_categoria",
    "cd_criticidade",
    "flag_qualidade",
    "flag_id_valido",
    "flag_nome_valido",
    "flag_categoria_valida",
    "flag_criticidade_valida"
).limit(10))

## 4️⃣ Separação de Dados Válidos e Inválidos

In [0]:
df_invalidos = (
    df_validacao
    .filter(F.col("flag_qualidade") == "ERRO")
    .withColumn(
        "motivo_rejeicao",
        F.concat_ws(
            "; ",
            F.when(~F.col("flag_id_valido"), F.lit("cd_motivo NULL")),
            F.when(~F.col("flag_nome_valido"), F.lit("ds_motivo vazio")),
            F.when(~F.col("flag_categoria_valida"), F.lit("cd_categoria fora do range (0-5)")),
            F.when(~F.col("flag_criticidade_valida"), F.lit("cd_criticidade inválida (deve ser 1, 2 ou 3)"))
        )
    )
    .withColumn("dt_validacao", F.lit(processing_timestamp))
    .drop(
        "flag_id_valido",
        "flag_nome_valido",
        "flag_categoria_valida",
        "flag_criticidade_valida",
        "data_processamento"
    )
)

In [0]:
# ---------------------------
# Estatísticas de qualidade
# ---------------------------
total_validos = df_validos.count()
total_invalidos = df_invalidos.count()
total_processado = total_validos + total_invalidos

taxa_qualidade = (total_validos * 100.0 / total_processado) if total_processado > 0 else 0

print("="*80)
print("📊 RESULTADO DA VALIDAÇÃO")
print("="*80)
print(f"\nTotal processado: {total_processado:,}")
print(f"✅ Registros válidos: {total_validos:,} ({taxa_qualidade:.1f}%)")
print(f"❌ Registros inválidos: {total_invalidos:,} ({100-taxa_qualidade:.1f}%)")

if taxa_qualidade < 90:
    print(f"\n⚠️  ALERTA: Taxa de qualidade abaixo de 90%!")
    print(f"   Investigar problemas na origem dos dados")
elif total_invalidos > 0:
    print(f"\n⚠️  {total_invalidos} registro(s) rejeitado(s) - verificar tb_motivo_invalidos")
else:
    print(f"\n🎉 100% de qualidade - nenhum registro rejeitado!")

print("="*80)

In [0]:
# ---------------------------
# Análise de erros (se houver inválidos)
# ---------------------------
if total_invalidos > 0:
    print("\n📋 DISTRIBUIÇÃO DE ERROS:")
    print("="*80)
    
    df_erros_dist = (df_invalidos
        .groupBy("motivo_rejeicao")
        .count()
        .withColumn("percentual", F.round((F.col("count") * 100.0 / total_invalidos), 2))
        .orderBy(F.desc("count"))
    )
    
    display(df_erros_dist)
    
    # Mostrar amostras de registros inválidos
    print("\n📋 AMOSTRA DE REGISTROS INVÁLIDOS:")
    display(df_invalidos.select(
        "cd_motivo",
        "ds_motivo",
        "cd_categoria",
        "cd_criticidade",
        "flag_qualidade",
        "motivo_rejeicao"
    ).limit(5))

## 5️⃣ Gravação: MERGE de Registros Válidos

In [0]:
# ---------------------------
# MERGE registros válidos na tabela principal
# ---------------------------
if total_validos > 0:
    # Criar temp view
    df_validos.createOrReplaceTempView("temp_motivos_validos")
    
    print(f"🔄 Executando MERGE de {total_validos} registros válidos em {TABLE_TARGET_VALIDOS}...")
    
    # MERGE SQL
    spark.sql(f"""
        MERGE INTO {TABLE_TARGET_VALIDOS} AS target
        USING temp_motivos_validos AS source
        ON target.cd_motivo = source.cd_motivo
        
        WHEN MATCHED THEN
            UPDATE SET
                target.ds_motivo = source.ds_motivo,
                target.cd_categoria = source.cd_categoria,
                target.cd_criticidade = source.cd_criticidade,
                target.dt_ingestion = source.dt_ingestion,
                target.dc_origem = source.dc_origem
        
        WHEN NOT MATCHED THEN
            INSERT (
                cd_motivo,
                ds_motivo,
                cd_categoria,
                cd_criticidade,
                dt_ingestion,
                dc_origem
            )
            VALUES (
                source.cd_motivo,
                source.ds_motivo,
                source.cd_categoria,
                source.cd_criticidade,
                source.dt_ingestion,
                source.dc_origem
            )
    """)
    
    print(f"✅ MERGE concluído com sucesso!")
    
    # Mostrar estatísticas do MERGE
    merge_stats = spark.sql(f"""
        SELECT 
            version,
            timestamp,
            operationMetrics.numTargetRowsInserted as inseridos,
            operationMetrics.numTargetRowsUpdated as atualizados,
            operationMetrics.numOutputRows as total_afetados
        FROM (DESCRIBE HISTORY {TABLE_TARGET_VALIDOS})
        WHERE operation = 'MERGE'
        ORDER BY version DESC
        LIMIT 1
    """)
    
    if merge_stats.count() > 0:
        stats = merge_stats.collect()[0]
        print(f"\n📊 Estatísticas do MERGE:")
        print(f"   - Registros inseridos: {stats['inseridos']}")
        print(f"   - Registros atualizados: {stats['atualizados']}")
        print(f"   - Total afetados: {stats['total_afetados']}")
else:
    print("⚠️  Nenhum registro válido para inserir na Silver")

## 6️⃣ Gravação: APPEND de Registros Inválidos

In [0]:
# ---------------------------
# APPEND registros inválidos na tabela de auditoria
# ---------------------------
if total_invalidos > 0:
    print(f"📝 Enviando {total_invalidos} registro(s) inválido(s) para auditoria...")
    
    # APPEND (acumula histórico de rejeições)
    (df_invalidos.write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_TARGET_INVALIDOS))
    
    print(f"✅ Registros inválidos gravados em {TABLE_TARGET_INVALIDOS}")
    print(f"\n💡 Use a query abaixo para investigar:")
    print(f"""
    SELECT 
        motivo_rejeicao,
        COUNT(*) as total
    FROM {TABLE_TARGET_INVALIDOS}
    WHERE DATE(dt_validacao) = CURRENT_DATE()
    GROUP BY motivo_rejeicao
    """)
else:
    print("🎉 Nenhum registro inválido - 100% de qualidade!")

## 7️⃣ Verificações Finais

In [0]:
# ---------------------------
# Verificar tabela de válidos
# ---------------------------
df_silver_final = spark.table(TABLE_TARGET_VALIDOS)
total_silver = df_silver_final.count()

print("="*80)
print("✅ VERIFICAÇÕES FINAIS")
print("="*80)

print(f"\n📊 {TABLE_TARGET_VALIDOS}")
print(f"   Total de registros: {total_silver:,}")

# Distribuição por categoria
print(f"\n📊 Distribuição por Categoria:")
df_silver_final.groupBy("cd_categoria").count().orderBy("cd_categoria").show()

# Distribuição por criticidade
print(f"\n📊 Distribuição por Criticidade:")
df_silver_final.groupBy("cd_criticidade").count().orderBy("cd_criticidade").show()

In [0]:
# ---------------------------
# Verificar tabela de inválidos (se existir)
# ---------------------------
try:
    df_invalidos_total = spark.table(TABLE_TARGET_INVALIDOS)
    total_invalidos_historico = df_invalidos_total.count()
    
    print(f"\n📊 {TABLE_TARGET_INVALIDOS}")
    print(f"   Total de registros (histórico): {total_invalidos_historico:,}")
    
    # Últimas rejeições
    print(f"\n📊 Últimas 5 rejeições:")
    display(df_invalidos_total.orderBy(F.desc("dt_validacao")).limit(5))
    
except:
    print(f"\n📊 {TABLE_TARGET_INVALIDOS}")
    print(f"   Tabela ainda não existe (nenhum registro rejeitado)")

## 8️⃣ Métricas de Qualidade

In [0]:
print("="*80)
print("📈 MÉTRICAS DE QUALIDADE DE DADOS")
print("="*80)

# KPIs
print(f"\n🎯 KPIs do Processamento:")
print(f"   - Taxa de Sucesso: {taxa_qualidade:.2f}%")
print(f"   - Taxa de Rejeição: {100-taxa_qualidade:.2f}%")
print(f"   - Registros Processados: {total_processado:,}")

# Alertas
if taxa_qualidade < 80:
    print(f"\n🚨 CRÍTICO: Taxa de qualidade muito baixa ({taxa_qualidade:.1f}%)")
    print(f"   Ação requerida: Investigar urgentemente a origem dos dados")
elif taxa_qualidade < 95:
    print(f"\n⚠️  ATENÇÃO: Taxa de qualidade abaixo do esperado ({taxa_qualidade:.1f}%)")
    print(f"   Ação recomendada: Revisar regras de validação")
else:
    print(f"\n✅ EXCELENTE: Taxa de qualidade acima de 95%")

print("\n" + "="*80)

## 9️⃣ Queries de Exemplo para Uso

In [0]:
%sql
-- Query 1: Ver todos os motivos válidos com categoria e criticidade
SELECT 
    m.cd_motivo,
    m.ds_motivo,
    cat.ds_categoria,
    crit.ds_criticidade,
    m.cd_criticidade
FROM v_credit.silver.tb_motivo m
JOIN v_credit.silver.tb_categoria cat 
    ON m.cd_categoria = cat.cd_categoria
JOIN v_credit.silver.tb_criticidade crit
    ON m.cd_criticidade = crit.cd_criticidade
ORDER BY cat.cd_categoria, m.cd_motivo
LIMIT 10;

In [0]:
%sql
SELECT 
    cat.cd_categoria,
    crit.ds_criticidade,
    COUNT(*) as total_motivos
FROM v_credit.silver.tb_motivo m
JOIN v_credit.silver.tb_categoria cat 
    ON m.cd_categoria = cat.cd_categoria
JOIN v_credit.silver.tb_criticidade crit
    ON m.cd_criticidade = crit.cd_criticidade
WHERE cat.cd_categoria > 0
GROUP BY cat.cd_categoria, crit.ds_criticidade
ORDER BY cat.cd_categoria, crit.ds_criticidade;

In [0]:
%sql
-- Query 3: Ver registros inválidos (se houver)
SELECT 
    cd_motivo,
    ds_motivo,
    flag_qualidade,
    motivo_rejeicao,
    dt_validacao
FROM v_credit.silver.tb_motivo_invalidos
ORDER BY dt_validacao DESC
LIMIT 10;

## ✅ Processamento Concluído!

**Resumo**:
- ✅ Dados lidos da Bronze
- ✅ Limpeza e padronização aplicadas
- ✅ Validações de qualidade executadas
- ✅ Registros válidos inseridos via MERGE em `tb_motivo`
- ✅ Registros inválidos arquivados em `tb_motivo_invalidos`
- ✅ Métricas de qualidade calculadas

**Próximos Passos**:
1. Investigar registros inválidos (se houver)
2. Corrigir na origem (Bronze)
3. Reprocessar pipeline
4. Avançar para camada Gold

In [0]:
# Log final
print(f"\n{'='*80}")
print(f"✅ PROCESSAMENTO CONCLUÍDO")
print(f"{'='*80}")
print(f"Tabela principal: {TABLE_TARGET_VALIDOS} ({total_silver:,} registros)")
print(f"Tabela auditoria: {TABLE_TARGET_INVALIDOS}")
print(f"Taxa de qualidade: {taxa_qualidade:.2f}%")
print(f"{'='*80}")